In [2]:
import pandas as pd

In [6]:
bx_ratings = pd.read_csv("newData/df_ratings.csv")
bx_books = pd.read_csv("newData/df_bookss.csv")
bx_users = pd.read_csv("newData/df_userss.csv")

In [7]:
bx_users.head()

,User-ID,Location,Age
0,99,usa,42
1,114,usa,57
2,165,usa,62
3,243,usa,37
4,244,usa,37


In [10]:
countries = set( bx_users['Location'])
print len(countries)
bx_users['Location'].value_counts()

73


 usa                    6941
 canada                  900
Global                   413
 united kingdom          279
 germany                 156
 australia               129
 spain                    67
 portugal                 65
 netherlands              53
 france                   50
 malaysia                 48
 italy                    34
 new zealand              29
 finland                  18
 switzerland              18
 austria                  18
 singapore                17
 romania                  13
 ireland                  12
 sweden                   11
 philippines              10
 brazil                   10
 japan                     9
 mexico                    8
 norway                    6
 china                     6
 iran                      5
 poland                    5
 belgium                   5
 kuwait                    4
                        ... 
 south korea               2
 bulgaria                  2
 russia                    1
 czech republi

In [11]:
bx_ratings.head()

,User-ID,ISBN,Book-Rating
0,276925,002542730X,10.0
1,276925,60520507,9.0
2,276925,151010633,9.0
3,276925,316666343,9.0
4,276925,345391810,9.0


In [16]:
map_user_country = {}
age_user_map = {}
for i,item in bx_users.iterrows():
    map_user_country[ item['User-ID'] ] = item['Location']
    age_user_map[item['User-ID'] ] = item['Age']
len(map_user_country)

9403

In [23]:
len( set( bx_ratings['User-ID']))
gg = set()

In [18]:
book_country_map = [{}, {}, {}]

for i,item in bx_ratings.iterrows():
    book = item['ISBN']
    user = item['User-ID']
    ratings = item['Book-Rating']
    country = map_user_country[user]
    group = 0

    if( age_user_map[user] <= 30 ):
        group = 0
    elif(age_user_map[user] <= 60  and age_user_map[user] > 30 ):
        group = 1
    else:
        group = 2
    if( book_country_map[group].get(country) == None):
        book_country_map[group][country] = [(book, ratings)]
    else:
        book_country_map[group][country].append( (book, ratings) )
    

In [117]:
import operator
def find_best_books(listofBooks, cnt):    
    book_rate = {}
    book_count = {}
    for book,rate in listofBooks:
        if( book_rate.get(book) == None):
            book_rate[book] = 0
            book_count[book] = 0
        book_rate[book] += float(rate)
        book_count[book] += 1
    best_books = []
    limit = ( max(book_count.iteritems(), key=operator.itemgetter(1))[1] ) * 0.2
    for keys in book_count.iterkeys():
        if( book_count[keys] >= limit):
            best_books.append( ( round(book_rate[keys]/book_count[keys] , 2) , keys))
    #print book_count['439139597']
    cnt = min(cnt, len(best_books) )
    return sorted(best_books, reverse=True)[:cnt]
    

In [118]:
find_best_books( book_country_map[0][' usa'])

TypeError: find_best_books() takes exactly 2 arguments (1 given)

In [119]:
def getUserAgeMap():
    map_user_age = {}
    bx_users = pd.read_csv("newData/df_userss.csv")
    for i,item in bx_users.iterrows():
        map_user_age[ item['User-ID'] ] = item['Age']
    return map_user_age

def getUserCountryMap():
    map_user_country = {}
    bx_users = pd.read_csv("newData/df_userss.csv")
    for i,item in bx_users.iterrows():
        map_user_country[ item['User-ID'] ] = item['Location']
    return map_user_country

In [120]:
def getAgeGroup( age ):
    if( age <= 30):
        return 0
    elif ( age > 30 and age <= 60):
        return 1
    return 2

In [124]:
def getDictionaryOfGroupByBooks( user ):
    ageMap = getUserAgeMap()
    countryMap = getUserCountryMap()
    
    group = getAgeGroup( ageMap[user] )
    country = countryMap[ user ]
    
    bx_ratings = pd.read_csv("newData/df_ratings.csv")
    listOfBooks = []
    for i,item in bx_ratings.iterrows():
        user = item['User-ID']
        if( countryMap[ user ] == country and getAgeGroup( ageMap[ user ] ) == group ):
            listOfBooks.append( (item['ISBN'], item['Book-Rating']) )
    return listOfBooks

In [125]:
def getRecommendationByDemographic(user):
    listofBooks = getDictionaryOfGroupByBooks(user)
    recommended_books = find_best_books(listofBooks, 5)
    return recommended_books

In [126]:
getRecommendationByDemographic(276925)

[(9.0, '751503894'),
 (8.6, '8478884459'),
 (8.33, '8495618605'),
 (8.33, '8477204055'),
 (8.33, '345350499')]